# Incremental Matrix Profiles for Streaming Time Series Data

Now that you have a basic understanding of how to compute a matrix profile, in this short tutorial, we will demonstrate how to incrementally update your matrix profile when you have streaming (on-line) data using the `stumpy.stumpi` ("STUMP Incremental") function. You can learn more about the details of this approach by reading Section G of the [Matrix Profile I](https://www.cs.ucr.edu/~eamonn/PID4481997_extend_Matrix%20Profile_I.pdf) paper and Section 4.6 and Table 5 [this paper](https://www.cs.ucr.edu/~eamonn/ten_quadrillion.pdf).

## Getting Started

Let's import the packages that we'll need to create and analyze a randomly generated time series data set.

In [1]:
import numpy as np
import stumpy
import numpy.testing as npt
import time

## Generating Some Random Time Series Data

Imagine that we have an [IoT](https://en.wikipedia.org/wiki/Internet_of_things) sensor that has been collecting data once an hour for the last 14 days. That would mean that we've amassed `14 * 24 = 336` data points up until this point and our data set might look like this:

In [2]:
T = np.random.rand(336)

And, perhaps, we know from experience that an interesting motif or anomaly might be detectable within a 12 hour (sliding) time window:

In [3]:
m = 12

## Typical Batch Analysis

To compute the matrix profile using a batch process is straightforward using `stumpy.stump`:

In [4]:
mp = stumpy.stump(T, m)

But as the length of `T` grows with each passing hour, it will take increasingly more time to compute the matrix profile since `stumpy.stump` will actually re-compute all of the pairwise distances between all subsequences within the time series. This is super time consuming! Instead, for streaming data, we want to find a way to take the new incoming (single) data point and compare the subsequence that it resides in with the rest of the time series (i.e., compute the distance profile) and update the existing matrix profile. Luckily, this can be easily accomplished with `stumpy.stumpi` or "STUMP Incremental".

## Streaming (On-line) Analysis with STUMPI

As we wait for the next data point, `t`, to arrive, we can take our existing data set and initialize a set of inputs that are necessary for incrementally updating our matrix profile:

In [5]:
T, P, I, QT, M_T, Σ_T = stumpy.stumpi_init(T, m)

Here: 

* `T` is our original time series but with all `nan`/`inf` values set to zero (`nan`/`inf` are not supported)
* `P` is the matrix profile
* `I` is the matrix profile index
* `QT` is the sliding window dot product for the last subsequence in `T`
* `M_T` is the sliding window mean for all subsequences in `T` 
* `Σ_T` is the sliding window standard deviation for all subsequences in `T`

And when a new data point, `t`, arrives:

In [6]:
t = np.random.rand()

We can call the `stumpy.stumpi` function to quickly and easily update the matrix profile, `P`, and matrix profile indices, `I`:

In [7]:
T, P, I, QT, M_T, Σ_T = stumpy.stumpi(t, T, m, P, I, QT, M_T, Σ_T)

In fact, with each `t`, all of the values that are returned by `stumpy.stumpi` (i.e., `T, P, I, QT, M_T, Σ_T`) will also increase in length by a single element in order to accomodate the newly added subsequence. For example, if we look at the length of, say, `T`:

In [8]:
len(T)

337

notice that the length of `T` has increased from `336` to `337` since `t` has been appended to the time series and all of the other inputs have been updated accordingly as well. So, not only does `stumpy.stumpi` compare the new subsequence with all of the existing ones and updates the historical values but it also determines which one of the existing subsequences is the nearest neighbor to the new subsequence and appends this information to the matrix profile. And this can continue on, say, for another 1,000 iterations (or indefinitely) as additional data is streamed in:

In [9]:
for i in range(1000):
    t = np.random.rand()
    T, P, I, QT, M_T, Σ_T = stumpy.stumpi(t, T, m, P, I, QT, M_T, Σ_T)

It is important to reiterate that incremental `stumpy.stumpi` is different from batch `stumpy.stump` in that it does <b><u>not</u></b> waste any time re-computing any of the past pairwise distances. `stumpy.stumpi` only spends time computing <b><u>new</u></b> distances and then updates the appropriate arrays where necessary and, thus, it is really fast!

## Validation

### The Matrix Profile

Now, this claim of "fast updating" with streaming (on-line) data may feel strange or seem magical so, first, let's validate that the output from incremental `stumpy.stumpi` is the same as performing batch `stumpy.stump`. Let's start with the full time series with `64` data points and compute the full matrix profile:

In [10]:
T_full = np.random.rand(64)
m = 8

mp = stumpy.stump(T_full, m)
P_full = mp[:, 0]
I_full = mp[:, 1]

Next, for `stumpy.stumpi`, we'll only start with the first `10` elements from the full length time series and then incrementally stream in the additional data points one at a time:

In [11]:
# Start with half of the full length time series and initialize inputs
T_stream = T_full[:10].copy()
T_stream, P_stream, I_stream, QT_stream, M_T_stream, Σ_T_stream = stumpy.stumpi_init(T_stream, m)

# Incrementally add one new data point at a time until `T_stream == T_full` and update the matrix profile
for i in range(len(T_stream), len(T_full)):
    t = T_full[i]
    T_stream, P_stream, I_stream, QT_stream, M_T_stream, Σ_T_stream = stumpy.stumpi(t, T_stream, m, P_stream, I_stream, QT_stream, M_T_stream, Σ_T_stream)

Now that we're done, let's check and validate that:

1. `T_stream == T_full`
2. `P_stream == P_full`
3. `I_stream == I_full`

In [12]:
npt.assert_almost_equal(T_stream, T_full)
npt.assert_almost_equal(P_stream, P_full)
npt.assert_almost_equal(I_stream, I_full)

There are no errors! So, this means that `stump.stumpi` indeed produces the correct matrix profile results that we'd expect.

### The Performance

We've basically claimed that incrementally updating our matrix profile with `stumpy.stumpi` is much faster (in total computation time) than performing a full pairwise distance calculation with `stumpy.stump` as each new data point arrives. Let's actually compare the timings by taking a full time series that is 1,000 data points in length and we initialize both approaches with the first 20% of the time series (i.e., the first 200 points) and append a single new data point at each iteration before re-computing the matrix profile:

In [13]:
T_full = np.random.rand(1000)
T_stream = T_full[:200].copy()
m = 10

# `stumpy.stump` timing
start = time.time()
mp = stumpy.stump(T_stream, m)
for i in range(200, len(T_full)):
    T_stream = np.append(T_stream, T_full[i])
    mp = stumpy.stump(T_stream, m)
stump_time = time.time() - start

# `stumpy.stumpi` timing
start = time.time()
T_stream, P, I, QT, M_T, Σ_T = stumpy.stumpi_init(T_stream, m)
for i in range(200, len(T_full)):
    t = T_full[i]
    T_stream, P, I, QT, M_T, Σ_T = stumpy.stumpi(t, T_stream, m, P, I, QT, M_T, Σ_T)
stumpi_time = time.time() - start

print(f"stumpy.stump: {np.round(stump_time,1)}s")
print(f"stumpy.stumpi:, {np.round(stumpi_time, 1)}s")

stumpy.stump: 460.8s
stumpy.stumpi:, 2.6s


Setting aside the fact that having more CPUs will speed up both approaches, we clearly see that incremental `stumpy.stumpi` (`2.6s` of total computational time) is several orders of magnitude faster than batch `stumpy.stump` (`460s` of total computational time) for processing streaming data. In fact for the current hardware, on average, it is taking roughly `460.8s / 800 = 0.576s` for `stumpy.stump` to analyze each new matrix profile. So, if you have a new data point arriving every once every half a second, then you wouldn't be able to keep up. However, with an average analysis time of `2.6s / 800 = 0.00325s`, `stumpy.stumpi` should be able to comfortably handle and process ~300 new data points per second using fairly modest hardware. Additionally, batch `stumpy.stump` will get even slower as more and more data points get appended to the existing time series while `stumpy.stumpi` will continue to be highly performant. 

## Caveats

As with all software, there are a few things to keep in mind when using `stumpy.stumpi`: 

1. There is currently no support for time series with that contain `NaN`/`inf` values
2. This only works for self-joins and not AB-joins
3. Removing data points from the front/head of the time series should be avoided since the matrix profile indices are recorded based on the absolute subsequence position within the time series 

## Summary

And that's it! You've just learned how to incrementally update your matrix profile for streaming (on-line) data. 

## Resources
​
[Matrix Profile I](https://www.cs.ucr.edu/~eamonn/PID4481997_extend_Matrix%20Profile_I.pdf)

[Time Series Joins, Motifs, Discords and Shapelets:
A Unifying View that Exploits the Matrix Profile](https://www.cs.ucr.edu/~eamonn/MP_journal.pdf) (see Section 4.6 and Table 5)